In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
# model that does best across both
gen_metrics = pd.read_csv('FINAL_GEN_METRICS_143.csv')
nor_metrics = pd.read_csv('FINAL_DENOISE_METRICS_143.csv')
combined = pd.read_csv('FINAL_COMBINED_METRICS_143.csv')

corr_gen_rank = combined['corr_gen'].rank(ascending=False)
corr_nor_rank = combined['corr_nor'].rank(ascending=False)
combined['corr_combined_rank'] = corr_gen_rank + corr_nor_rank
combined = combined.sort_values(by='corr_combined_rank')
combined

In [ ]:
combined['num_params_nor_rank'] = combined['num_params'].rank(ascending=False)
top3_models = combined[(combined['num_params_nor_rank'] <= 3)]
top3_models

In [ ]:
dataset_config='denoise'

if dataset_config=='denoise':
    df = nor_metrics
elif dataset_config=='gen':
    df = gen_metrics

df.sort_values(by='corr', inplace=True, ascending=False)
df

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# to see shorter ranges of parameters, set PARAM_MAX to 10k

PARAM_MAX = 80000
filt_df = df[df['num_params'] <= PARAM_MAX]

params = filt_df['num_params']
rrmses = filt_df['rrmse']
corr = filt_df['corr']
sdr = filt_df['sdr']

def plot_trends(params, rrmses, corr, sdr, name):
    # only plots correlation + log and loglog
    plt.figure(dpi=300)
    plt.scatter(params, corr, s=5)
    plt.xlabel('Number of Parameters')
    plt.ylabel('CC')
    plt.title(f'{name}')
    plt.show()

    plt.figure(dpi=300)
    plt.scatter(np.log(params.astype(float)), corr, s=5)
    plt.xlabel('log(params)')
    plt.ylabel('CC')
    plt.title(f'{name}')
    plt.show()

    plt.figure(dpi=300)
    plt.scatter(np.log(np.log((params.astype(float)))), corr, s=5)
    plt.xlabel('loglog(params)')
    plt.ylabel('CC')
    plt.title(f'{name}')
    plt.show()




def plot_trends_all(params, rrmses, corr, sdr, name):
    # plots all metrics v params
    plt.figure()
    plt.scatter(params, rrmses, s=5)
    plt.xlabel('Number of Parameters')
    plt.ylabel('T-RRMSE')
    plt.title(f'{name}')
    plt.show()

    plt.figure()
    plt.scatter(np.log(np.log(np.array(params))), rrmses, s=5)
    plt.xlabel('loglog(params)')
    plt.ylabel('T-RRMSE')
    plt.title(f'{name}')
    plt.show()

    plt.figure()
    plt.scatter(params, corr, s=5)
    plt.xlabel('Number of Parameters')
    plt.ylabel('CC')
    plt.title(f'{name}')
    plt.show()

    plt.figure()
    plt.scatter(params, sdr, s=5)
    plt.xlabel('Number of Parameters')
    plt.ylabel('SDR')
    plt.title(f'{name}')
    plt.show()

# plot_trends(params, rrmses, corr, sdr, 'CC vs Num Params')

The next function gets the best fit for any metric v plot distribution. You can add custom models as needed.

In [ ]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_squared_error

def loglog_model(x, c, b, d):
    return c * np.log(np.log(x + b)) + d

def log_model(x, a, b, c):
    return a * np.log(x + b) + c

def sigmoid_model(x, L, k, x0, c):
    return L / (1 + np.exp(-k * (x - x0))) + c

def exp_model(x, a, b, c):
    return a * (1 - np.exp(-b * x)) + c

def exp_sqrt_model(x, a, b, c, p):
    return a * (1 - np.exp(-b * (x**p))) + c

def hyperbolic(x, a, b, c):
    return c - a / (x + b)

def hill(x, a, c, k, p):
    return c - a/(1 + (x/k)**p)

def fit_trend(params, ys):

    x = params.astype(float).values
    y = ys
    results = {}

    # loglog
    popt_loglog, _ = curve_fit(loglog_model, x, y, p0=(1.0, 1.0, 0.0),
                               bounds=((-np.inf, 0, -np.inf), (np.inf, np.inf, np.inf)),
                               maxfev=20000)
    y_pred = loglog_model(x, *popt_loglog)
    results["loglog"] = (popt_loglog, r2_score(y, y_pred), mean_squared_error(y, y_pred))

    # log
    popt_log, _ = curve_fit(log_model, x, y, p0=(1.0, 1.0, 0.0),
                            bounds=((-np.inf, 0, -np.inf), (np.inf, np.inf, np.inf)),
                            maxfev=20000)
    y_pred = log_model(x, *popt_log)
    results["log"] = (popt_log, r2_score(y, y_pred), mean_squared_error(y, y_pred))

    # sigmoid
    p0 = [1.0, 0.001, np.median(x), min(y)]
    popt_sig, _ = curve_fit(sigmoid_model, x, y, p0=p0, maxfev=20000)
    y_pred = sigmoid_model(x, *popt_sig)
    results["sigmoid"] = (popt_sig, r2_score(y, y_pred), mean_squared_error(y, y_pred))

    # 1-exp
    # p0 = [1.0, 0.001, min(y)]
    # popt_exp, _ = curve_fit(exp_model, x, y, p0=p0, maxfev=20000)
    # y_pred = exp_model(x, *popt_exp)
    # results["1-exp"] = (popt_exp, r2_score(y, y_pred), mean_squared_error(y, y_pred))

    # # exp sqrt model
    # p0 = [1.0, 0.001, min(y), 0.5]
    # popt_exp_sqrt, _ = curve_fit(exp_sqrt_model, x, y, p0=p0, maxfev=20000)
    # y_pred = exp_sqrt_model(x, *popt_exp_sqrt)
    # results["1-exp-sqrt"] = (popt_exp_sqrt, r2_score(y, y_pred), mean_squared_error(y, y_pred))

    # hyperbolic
    p0 = [1, 1, 1]
    popt_hyp, _ = curve_fit(hyperbolic, x, y, p0=p0, maxfev=20000)
    y_pred = hyperbolic(x, *popt_hyp)
    results["hyperbolic"] = (popt_hyp, r2_score(y, y_pred), mean_squared_error(y, y_pred))

    # hill
    p0 = [1, 1, 1, 1]
    popt_hill, _ = curve_fit(hill, x, y, p0, maxfev=20000)
    y_pred = hill(x, *popt_hill)
    results["hill"] = (popt_hill, r2_score(y, y_pred), mean_squared_error(y, y_pred))

    # print summary
    for k, (p, r2, mse) in results.items():
        print(f"{k}: params={p}, R2={r2:.4f}, MSE={mse:.6f}")

    # choose best by R2
    best_name = max(results.items(), key=lambda kv: kv[1][1])[0]
    best_name = str(best_name)
    print(f"Best model: {best_name}")
    return results, best_name


In [ ]:
# plot the best fit
def plot_trend_lines(params, ys, results, best_name, name):
    x = params.astype(float).values
    y = ys
    # y = ys.astype(float).values
    x_fit = np.linspace(min(x), max(x), 1000)

    plt.figure(dpi=300)
    plt.scatter(x, y, s=5, label="Data")

    if best_name == "loglog":
        y_fit = loglog_model(x_fit, *results["loglog"][0])
    elif best_name == "log":
        y_fit = log_model(x_fit, *results["log"][0])
    elif best_name == "sigmoid":
        y_fit = sigmoid_model(x_fit, *results["sigmoid"][0])
    elif best_name == "1-exp":
        y_fit = exp_model(x_fit, *results["1-exp"][0])
    elif best_name == "1-exp-sqrt":
        y_fit = exp_sqrt_model(x_fit, *results["1-exp-sqrt"][0])
    elif best_name == "hyperbolic":
        y_fit = hyperbolic(x_fit, *results["hyperbolic"][0])
    elif best_name == "hill":
        y_fit = hill(x_fit, *results["hill"][0])
    

    # change this manually according to the plot

    plt.plot(x_fit, y_fit, color="red", label=f"Best Fit")
    # a * (1 - np.exp(-b * (x**p))) + c
    # plt.text(0.65, 0.25, r"$y = c + \alpha\left[1-\exp\left({\beta x^{-p}}\right)\right]$" +"\n"+"$R^2 = 0.726$", transform=plt.gca().transAxes, fontsize=10,
    #          verticalalignment='top')
    plt.text(0.75, 0.8, r"$y = c - \frac{\alpha}{1+(x/k)^p}$" + "\n\n" + r"$R^2 = 0.732$", transform=plt.gca().transAxes, fontsize=10,
            verticalalignment='top')
    plt.xlabel("Number of Parameters")
    plt.ylabel("CC")
    plt.title(f"{name} with Best Fit Curve")
    plt.legend()
    plt.show()
    # c * np.log(np.log(x + b)) + d
    # c - a / (x + b)
    # c - a/(1 + (x/k)**p)
    # def sigmoid_model(x, L, k, x0, c):
    #   return L / (1 + np.exp(-k * (x - x0))) + c

results, best_name = fit_trend(params, corr)
plot_trend_lines(params, corr, results, best_name, "CC")

Heatmaps to see how model types and sizes affect performance

In [ ]:
import seaborn as sns
# make heatmaps? average of perf across 2 dimensions
def heatmap(dim1, dim2, val):
    # dim1 = df[dim1]
    # dim2 = df[dim2]

    g_df = df.groupby([dim1, dim2])[[val]].mean()
    g_pivot = g_df.reset_index().pivot(index=dim1, columns=dim2, values=val)
    # print(g_pivot)

    pivot = g_pivot.to_numpy()

    plt.figure(dpi=300)
    sns.heatmap(pivot, annot=True, fmt=".3f", yticklabels=g_pivot.index, xticklabels=g_pivot.columns, cmap='crest')
    plt.title(f"{val} across {dim1} × {dim2}")
    plt.ylabel(dim1)
    plt.xlabel(dim2)
    plt.show()

    # print(g_df_np_rrmse)

dims = ['model_type', 'chans', 'groups', 'scale']
metrics = ['rrmse', 'corr', 'sdr']

for d1 in dims:
    for d2 in dims:
        if d1 != d2:
            for m in metrics:
                print(f"Heatmap for {m} across {d1} and {d2}")
                heatmap(d1, d2, m)

In [ ]:
import seaborn as sns

dim2='groups'
dim1='chans'
val='corr'


g_df = df[df['chans'] < 16].groupby([dim1, dim2])[[val]].mean()
g_pivot = g_df.reset_index().pivot(index=dim1, columns=dim2, values=val)
# print(g_pivot)

pivot = g_pivot.to_numpy()

plt.figure(dpi=300)
sns.heatmap(pivot, annot=True, fmt=".3f", yticklabels=g_pivot.index, xticklabels=g_pivot.columns, cmap='crest')
plt.title(f"CC across num_channels × {dim2}")
plt.ylabel(dim1)
plt.xlabel(dim2)
plt.show()